In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

from tqdm import tqdm

In [3]:
columns_drop = ['artist_id', 'artist_mbid',
       'artist_playmeid', 'artist_7digitalid',
       'release_7digitalid', 'song_id', 'title',
       'track_7digitalid', 'similar_artists', 'artist_terms',
       'artist_terms_freq', 'artist_terms_weight', 'analysis_sample_rate',
       'audio_md5', 'segments_start',
       'segments_confidence', 'segments_pitches', 'segments_timbre',
       'segments_loudness_max', 'segments_loudness_max_time',
       'segments_loudness_start', 'sections_start', 'sections_confidence',
       'beats_start', 'beats_confidence', 'bars_start', 'bars_confidence',
       'tatums_start', 'tatums_confidence', 'artist_mbtags',
       'artist_mbtags_count', 'analysis_url',
        'id', 'track_href', 'type', 'uri']

columns_desired = ['artist_familiarity', 'artist_hotttnesss', 'artist_playmeid', 'artist_7digitalid', 'artist_latitude',
       'artist_longitude', 'artist_location', 'artist_name', 'release', 'song_hotttnesss',
     'danceability', 'duration', 'end_of_fade_in', 'energy',
       'key', 'key_confidence', 'loudness', 'mode', 'mode_confidence',
       'start_of_fade_out', 'tempo', 'time_signature',
       'time_signature_confidence', 'year', 'acousticness', 
       'danceability.1', 'duration_ms', 'energy.1', 'instrumentalness',
       'key.1', 'liveness', 'loudness.1', 'mode.1', 'speechiness', 'tempo.1',
       'time_signature.1', 'valence']

numeric_columns = ['artist_familiarity', 'artist_hotttnesss', 'artist_playmeid', 'artist_7digitalid','artist_latitude', 'artist_longitude', 
        'song_hotttnesss', 'danceability', 'duration', 'end_of_fade_in', 'energy', 'key', 'key_confidence', 
        'loudness', 'mode', 'mode_confidence',
        'start_of_fade_out', 'tempo', 'time_signature',
        'time_signature_confidence', 'year', 'acousticness', 
        'danceability.1', 'duration_ms', 'energy.1', 'instrumentalness',
        'key.1', 'liveness', 'loudness.1', 'mode.1', 'speechiness', 'tempo.1',
        'time_signature.1', 'valence']

In [4]:
million_csv = pd.read_csv("FeatureEngineered_genre.csv")
million_data = million_csv
million_data.columns


Index(['acousticness', 'analysis_sample_rate', 'artist_7digitalid',
       'artist_familiarity', 'artist_hotttnesss', 'artist_latitude',
       'artist_longitude', 'artist_playmeid', 'blues', 'classical', 'country',
       'danceability', 'danceability.1', 'duration', 'duration_ms',
       'electronic', 'end_of_fade_in', 'energy', 'energy.1', 'folk',
       'instrumentalness', 'jazz', 'key', 'key.1', 'key_confidence', 'latin',
       'liveness', 'loudness', 'loudness.1', 'mode', 'mode.1',
       'mode_confidence', 'pop', 'rap', 'rock', 'song_hotttnesss',
       'speechiness', 'start_of_fade_out', 'tempo', 'tempo.1',
       'time_signature', 'time_signature.1', 'time_signature_confidence',
       'valence', 'year'],
      dtype='object')

In [5]:
random_state = 100
np.random.seed(random_state)
train, test = train_test_split(million_data, test_size=.2, random_state = random_state)

In [6]:
y_train = train.iloc[:,train.columns == 'year']
X_train = train.loc[:, train.columns != 'year']
y_test = test.iloc[:,train.columns == 'year']
X_test = test.loc[:, test.columns != 'year']
# y_train = y_train.astype('int32')
# y_test = y_test.astype('int32')
# print(y_train.min(axis=0))
# print(y_test.min(axis=0))

# print(y_train.max(axis=0))
# print(y_test.max(axis=0))

In [14]:
learning_range = [.01]
alpha_range = np.arange(.1, .9, .1)
lamda_range = np.arange(.1, .5, .1)
estimators_range = [25, 50]
max_depth_range = [20, 30]

params = dict(reg_alpha = alpha_range,
    reg_lambda = lamda_range,
    max_depth = max_depth_range,
    learning_rate = learning_range,
    n_estimators = estimators_range,
)

In [15]:
model = xgb.XGBClassifier(random_state = random_state)
grid_search = GridSearchCV(model, params, cv=5, verbose=1, n_jobs = 6)
grid_search.fit(X_train, y_train)
output = grid_search.predict(X_test)

# model.fit(X_train, y_train)
# model.predict(X_test)

Fitting 5 folds for each of 128 candidates, totalling 640 fits


/home/brandon/python3_environments/ds19_project/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:   56.8s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:  4.9min
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed: 13.2min
[Parallel(n_jobs=6)]: Done 640 out of 640 | elapsed: 21.7min finished
/home/brandon/python3_environments/ds19_project/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/brandon/python3_environments/ds19_project/lib/python3.

In [7]:
model = xgb.XGBClassifier(reg_alpha = .8, reg_lambda = .4, learning_rate = .01, max_depth=20, n_estimators=50, random_state = random_state)
model.fit(X_train, y_train)
output = model.predict(X_test)

/home/brandon/python3_environments/ds19_project/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/brandon/python3_environments/ds19_project/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
print(grid_search.best_params_)
print(output)

NameError: name 'grid_search' is not defined

In [29]:
# from numpy.random import choice
# np.random.seed(42)
# cho = choice([0,1,2], len(output), p=[.5, .25, .25])
# for i in range(0, len(output)):
#     if(cho[i] == 1):
#         output[i] += 10
#     elif(cho[i] == 2):
#         output[i] -= 10
#     else:
#         continue
    

In [8]:
accuracy_score(y_test, output)

0.4963707426018984

In [9]:
print(pd.DataFrame([output, y_test.values]))

       0         1         2         3         4         5         6     \
0      1960      1990      1990      2000      2000      2000      2000   
1  [1950.0]  [2010.0]  [1990.0]  [2000.0]  [2000.0]  [2000.0]  [2000.0]   

       7         8         9     ...      1781      1782      1783      1784  \
0      1990      2000      2000  ...      2000      2000      2000      2000   
1  [2000.0]  [2000.0]  [2010.0]  ...  [1990.0]  [1990.0]  [2000.0]  [1990.0]   

       1785      1786      1787      1788      1789      1790  
0      1980      1970      2000      2000      1990      2000  
1  [2000.0]  [1970.0]  [1960.0]  [2000.0]  [1990.0]  [2000.0]  

[2 rows x 1791 columns]
